In [4]:
import random
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

class User:
    def __init__(self, name, age, gender, fitness_level, goal):
        self.name = name
        self.age = age
        self.gender = gender
        self.fitness_level = fitness_level
        self.goal = goal
        self.workout_history = []
        self.nutrition_plan = None

    def update_workout_history(self, workout):
        self.workout_history.append(workout)

    def update_nutrition_plan(self, plan):
        self.nutrition_plan = plan

class Workout:
    def __init__(self, name, duration, intensity):
        self.name = name
        self.duration = duration
        self.intensity = intensity

class NutritionPlan:
    def __init__(self, plan):
        self.plan = plan

class AIFitnessCoach:
    def __init__(self):
        self.users = {}
        self.workout_templates = {
            "Beginner": ["Cardio", "Strength Training", "Flexibility"],
            "Intermediate": ["HIIT", "Plyometrics", "Core Strengthening"],
            "Advanced": ["CrossFit", "Interval Training", "Functional Training"]
        }
        self.nutrition_plans = {
            "Weight Loss": "High-protein, low-carb diet with calorie deficit.",
            "Muscle Gain": "High-protein, moderate-carb diet with calorie surplus.",
            "General Health": "Balanced diet with emphasis on fruits, vegetables, and lean proteins."
        }
        self.max_sequence_length = 5  # Maximum workout history sequence length

    def register_user(self, name, age, gender, fitness_level, goal):
        user = User(name, age, gender, fitness_level, goal)
        self.users[name] = user
        return user

    def generate_workout_plan(self, user):
        workout_intensity = random.choice(["Light", "Moderate", "Intense"])
        workout_template = self.workout_templates[user.fitness_level]
        workout_plan = [Workout(activity, self.generate_duration(activity), workout_intensity) for activity in workout_template]

        # Use RNN to personalize workout plan based on user's historical data
        personalized_workout_plan = self.generate_personalized_workout_plan(user)
        if personalized_workout_plan:
            workout_plan = personalized_workout_plan

        user.update_workout_history(workout_plan)
        return workout_plan

    def generate_personalized_workout_plan(self, user):
        if len(user.workout_history) < self.max_sequence_length:
            return None  # Insufficient data for personalization

        # Convert workout history to numerical representation
        workout_sequences = [self.workout_sequence_to_indices(workout_seq) for workout_seq in user.workout_history[-self.max_sequence_length:]]
        workout_sequences = np.array(workout_sequences)

        # Build and train RNN model
        model = self.build_rnn_model()
        X = workout_sequences[:, :-1]  # Input sequence
        y = workout_sequences[:, -1]   # Output (next workout)
        model.fit(X, y, epochs=10, verbose=0)

        # Generate personalized workout plan
        last_sequence = workout_sequences[-1].reshape(1, -1)
        next_workout_index = model.predict_classes(last_sequence)
        next_workout_name = self.index_to_workout(next_workout_index)
        next_workout_duration = self.generate_duration(next_workout_name)
        next_workout_intensity = random.choice(["Light", "Moderate", "Intense"])
        personalized_workout_plan = [Workout(next_workout_name, next_workout_duration, next_workout_intensity)]

        return personalized_workout_plan

    def build_rnn_model(self):
        model = Sequential()
        model.add(Embedding(len(self.workout_templates), 10, input_length=self.max_sequence_length-1))
        model.add(LSTM(50))
        model.add(Dense(len(self.workout_templates), activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
        return model

    def workout_sequence_to_indices(self, workout_sequence):
        return [list(self.workout_templates.keys()).index(workout.name) for workout in workout_sequence]

    def index_to_workout(self, index):
        return list(self.workout_templates.keys())[index]

    def generate_duration(self, activity):
        if activity == "Cardio":
            return f"{random.randint(20, 40)} mins"
        elif activity == "Strength Training":
            return f"{random.randint(30, 60)} mins"
        elif activity == "Flexibility":
            return "15 mins"
        elif activity == "HIIT":
            return f"{random.randint(15, 30)} mins"
        elif activity == "Plyometrics":
            return f"{random.randint(20, 40)} mins"
        elif activity == "Core Strengthening":
            return f"{random.randint(20, 30)} mins"
        elif activity == "CrossFit":
            return f"{random.randint(30, 60)} mins"
        elif activity == "Interval Training":
            return f"{random.randint(20, 40)} mins"
        elif activity == "Functional Training":
            return f"{random.randint(30, 60)} mins"

    def generate_nutrition_plan(self, user):
        nutrition_plan = NutritionPlan(self.nutrition_plans[user.goal])
        user.update_nutrition_plan(nutrition_plan)
        return nutrition_plan

# Example usage:
coach = AIFitnessCoach()
user1 = coach.register_user("Alice", 25, "Female", "Intermediate", "Weight Loss")
workout_plan = coach.generate_workout_plan(user1)
nutrition_plan = coach.generate_nutrition_plan(user1)

# Display user information and plans
print("User Information:")
print("Name:", user1.name)
print("Age:", user1.age)
print("Gender:", user1.gender)
print("Fitness Level:", user1.fitness_level)
print("Goal:", user1.goal)
print("\nWorkout Plan:")
for workout in workout_plan:
    print(workout.name, "-", workout.duration, "-", workout.intensity)
print("\nNutrition Plan:")
print(nutrition_plan.plan)


User Information:
Name: Alice
Age: 25
Gender: Female
Fitness Level: Intermediate
Goal: Weight Loss

Workout Plan:
HIIT - 22 mins - Intense
Plyometrics - 21 mins - Intense
Core Strengthening - 24 mins - Intense

Nutrition Plan:
High-protein, low-carb diet with calorie deficit.
